<a href="https://colab.research.google.com/github/vipul43/image_description/blob/main/MutliLabelClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# installs

In [69]:
# imports
import scipy.io
import cv2
from google.colab.patches import cv2_imshow
from skimage import io
import numpy as np
import pandas as pd
from PIL import Image 
import matplotlib.pylab as plt
import pickle
from skimage import transform
from sklearn.model_selection import train_test_split
import tensorflow as tf
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, LancasterStemmer
import spacy
import nltk
import keras.backend as K

In [16]:
nltk.download('punkt') #tokenizer
nltk.download('wordnet') #lemmatization

lemmatizer = WordNetLemmatizer() #lemmatizer

sp = spacy.load('en_core_web_sm') #lexical importance find

ls = LancasterStemmer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
# data loading

!gdown --id 1mrjvJq6XNM8QAgajSgrVGpsj8Vrm3BEm #PASCAL50S

mat = scipy.io.loadmat('/content/pascal50S.mat')
print(type(mat))

Downloading...
From: https://drive.google.com/uc?id=1mrjvJq6XNM8QAgajSgrVGpsj8Vrm3BEm
To: /content/pascal50S.mat
100% 1.12M/1.12M [00:00<00:00, 74.0MB/s]
<class 'dict'>


In [55]:
classes = ['person',
           'bird', 
           'cat', 
           'cow', 
           'dog', 
           'horse', 
           'sheep', 
           'aeroplane', 
           'bicycle', 
           'boat', 
           'bus', 
           'car', 
           'motorbike', 
           'train', 
           'bottle', 
           'chair', 
           'dining table', 
           'potted plant',
           'sofa', 
           'tv'] #NOUNS

dict_classes = {'person':0, 'man':0, 'human':0,
           'bird':1, 'penguin':1, 'parrot':1, 'sparrow':1,
           'cat':2, 
           'cow':3, 
           'dog':4, 
           'horse':5,
           'sheep':6, 
           'aeroplane':7, 'airplane':7, 'flight':7, 'plane':7, 'jet':7,
           'bicycle':8, 'cycle':8,
           'boat':9, 'ship':9,
           'bus':10, 
           'car':11,
           'motorbike':12, 'bike':12, 'motorcycle':12,
           'train':13, 
           'bottle':14, 'waterbottle':14, 'drinkbottle':14, 'flask':14,
           'chair':15,
           'dining':16, 'diningtable': 16,
           'potted':17, 'plant':17, 'pottedplant': 17,
           'sofa':18, 'couch':18,
           'tv':19, 'monitor':19, 'television':19, 'desktop':19}

In [54]:
count = {'0':0,   #person
           '1':0, #bird
           '2':0, #cat
           '3':0, #cow
           '4':0, #dog
           '5':0, #horse
           '6':0, #sheep
           '7':0, #aeroplane
           '8':0, #bicycle
           '9':0, #boat
           '10':0, #bus
           '11':0, #car
           '12':0, #motorbike
           '13':0, #train
           '14':0, #bottle
           '15':0, #chair
           '16':0, #dining
           '17':0, #potted plant
           '18':0, #sofa
           '19':0} #tv/monitor

In [56]:
# observing data
data = []
idx=0
for sample in mat["train_sent_final"][0]:
  # image = io.imread(i[0][0])
  # cv2_imshow(image)
  link = [sample[0][0]] #image link
  cls = set()
  for k in sample[1]:
    for sent in k:
      # if idx==10:
      #   break
      # idx+=1

      for word in sent[0].split():
        pre_word = lemmatizer.lemmatize(ls.stem(word.lower()))
        if(pre_word in dict_classes.keys()):
          cls.add(dict_classes[pre_word])
  for cl in cls:
    count[str(cl)]+=1
  data.append([link, list(cls)])
print(data)

[[['https://filebox.ece.vt.edu/~vrama91/PASCAL/2008_006488.jpg'], [0, 9]], [['https://filebox.ece.vt.edu/~vrama91/PASCAL/2008_008632.jpg'], [11, 13]], [['https://filebox.ece.vt.edu/~vrama91/PASCAL/2008_003378.jpg'], [10, 11, 13]], [['https://filebox.ece.vt.edu/~vrama91/PASCAL/2008_008048.jpg'], [7]], [['https://filebox.ece.vt.edu/~vrama91/PASCAL/2008_002567.jpg'], []], [['https://filebox.ece.vt.edu/~vrama91/PASCAL/2008_004441.jpg'], [0, 11]], [['https://filebox.ece.vt.edu/~vrama91/PASCAL/2008_002467.jpg'], []], [['https://filebox.ece.vt.edu/~vrama91/PASCAL/2008_002395.jpg'], [4]], [['https://filebox.ece.vt.edu/~vrama91/PASCAL/2008_006397.jpg'], [0]], [['https://filebox.ece.vt.edu/~vrama91/PASCAL/2008_004621.jpg'], [6]], [['https://filebox.ece.vt.edu/~vrama91/PASCAL/2008_002222.jpg'], [0, 10, 13]], [['https://filebox.ece.vt.edu/~vrama91/PASCAL/2008_007519.jpg'], [0, 4]], [['https://filebox.ece.vt.edu/~vrama91/PASCAL/2008_006112.jpg'], [17]], [['https://filebox.ece.vt.edu/~vrama91/PASCAL

In [57]:
count

{'0': 315,
 '1': 56,
 '10': 60,
 '11': 139,
 '12': 0,
 '13': 68,
 '14': 0,
 '15': 107,
 '16': 0,
 '17': 50,
 '18': 77,
 '19': 61,
 '2': 64,
 '3': 59,
 '4': 83,
 '5': 0,
 '6': 57,
 '7': 57,
 '8': 0,
 '9': 59}

In [59]:
file = open("data.pkl", "wb")
pickle.dump(data, file)
file.close()

In [61]:
# preprocessing the dataset
'''
data -> url -> image -> array -> resized array
TrainX = array of images resized to (224x224x3)
TrainY = array of labels with size (20x1) in ones-zeros vector like [1, 1, 0, ....]
'''
# TrainX
new_shape = (224, 224, 3)
TrainX = []
for point in data:
  photo = io.imread(point[0][0])
  photo = transform.resize(image=photo, output_shape=new_shape)
  TrainX.append(photo)
TrainX = np.array(TrainX)

In [62]:
# TrainY
TrainY = []
for points in data:
  full_label = np.zeros(shape=(20, ))
  for label in points[1]:
    full_label[label] = 1
  TrainY.append(full_label)
TrainY = np.array(TrainY)

In [63]:
# data splitting
'''
train size: 700 images
val size: 300 images
output: TrainX, TrainY, ValX, ValY
'''
TrainX, ValX, TrainY, ValY = train_test_split(TrainX, TrainY, test_size=0.3, random_state=1)
print("Train X shape: ", TrainX.shape)
print("Train Y shape: ", TrainY.shape)
print("Val X shape: ", ValX.shape)
print("Val Y shape: ", ValY.shape)

Train X shape:  (700, 224, 224, 3)
Train Y shape:  (700, 20)
Val X shape:  (300, 224, 224, 3)
Val Y shape:  (300, 20)


In [98]:
# instantiate data augmentation
'''
use this 'tf.keras.preprocessing.ImageDataGenerator'
'''
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                                          shear_range=0.2, 
                                                          zoom_range=0.2,
                                                          rotation_range=15,
                                                          horizontal_flip=True)

In [99]:
def f1_macro(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

In [100]:
# model making
# input layer
input = tf.keras.Input(shape=(224, 224, 3), name='input')

# Transfer Learning with VGG16 model with weights as imagenet
vgg16 = tf.keras.applications.VGG16(include_top=False, weights="imagenet", classes=20)
vgg16.trainable = False
x = vgg16(input)

# Dense Layers
x = tf.keras.layers.Flatten(name='flatten')(x)
x = tf.keras.layers.BatchNormalization(name='norm1')(x)
x = tf.keras.layers.Dense(192, activation='relu', name='dense1')(x)
x = tf.keras.layers.BatchNormalization(name='norm2')(x)
x = tf.keras.layers.Dense(84, activation='relu', name='dense2')(x)
x = tf.keras.layers.BatchNormalization(name='norm3')(x)
x = tf.keras.layers.Dense(64, activation='relu', name='dense3')(x)
x = tf.keras.layers.BatchNormalization(name='norm4')(x)

#Output layer
output = tf.keras.layers.Dense(20, activation="sigmoid", name='output')(x)

model = tf.keras.models.Model(inputs=input, outputs=output, name='modelV1')

model.summary()

Model: "modelV1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 224, 224, 3)]     0         
_________________________________________________________________
vgg16 (Functional)           (None, None, None, 512)   14714688  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
norm1 (BatchNormalization)   (None, 25088)             100352    
_________________________________________________________________
dense1 (Dense)               (None, 192)               4817088   
_________________________________________________________________
norm2 (BatchNormalization)   (None, 192)               768       
_________________________________________________________________
dense2 (Dense)               (None, 84)                1621

In [101]:
# config cell
LR = 0.003
EPOCHS = 5
BATCH_SIZE = 16
DECAY = LR / EPOCHS
LOSS = tf.keras.losses.BinaryCrossentropy()
OPTIMIZER = tf.keras.optimizers.Adam(learning_rate=LR, decay=DECAY)
METRICS = [f1_macro]

In [102]:
# model compiling
model.compile(loss=LOSS, optimizer=OPTIMIZER, metrics=METRICS)

In [ ]:
# training
history = model.fit(datagen.flow(TrainX, TrainY, batch_size=BATCH_SIZE),
                  validation_data=(ValX/255., ValY/255.),
                  epochs=EPOCHS)

Epoch 1/5
44/44 [==============================] - 510s 12s/step - loss: 0.7306 - f1_macro: 0.1011 - val_loss: 1.6250 - val_f1_macro: 2.1151e-04
Epoch 2/5
15/44 [=========>....................] - ETA: 3:56 - loss: 0.4559 - f1_macro: 0.0742

In [ ]:
# plotting curves
'''
loss -> train, val
accuracy -> train, val
'''
plt.plot(list(history.history.values())[0])
plt.plot(list(history.history.values())[2])
plt.title('Loss Plot')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend(['train loss', 'val loss'])
plt.show()

plt.plot(list(history.history.values())[1])
plt.plot(list(history.history.values())[3])
plt.title('Accuracy Plot')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend(['train accuracy', 'val accuracy'])
plt.show()

In [ ]:
# random predictions on val dataset

rnd = np.random.randint(0, len(ValX))
point = ValX[rnd]
label = ValY[rnd]
pred = model.predict(np.asarray([point]))[0]
pred = (pred > 0.5)
pred = pred.astype(int)
print("True Label:", label)
print("Predicted Label:", pred)